# Data Preperation and Feature Engineering

# Under Contstruction! Coming soon!

# Continue learning

> [⏩ Next Module - Training and AutoML](https://raw.githubusercontent.com/dotnet/csharp-notebooks/fa302c12c7494e5f8a5fdbe5d8283d8ff1fb7009/machine-learning/03-Training%20and%20AutoML.ipynb)  
> [⏪ Last Module - Intro to Machine Learning](https://raw.githubusercontent.com/dotnet/csharp-notebooks/fa302c12c7494e5f8a5fdbe5d8283d8ff1fb7009/machine-learning/01-Intro%20to%20Machine%20Learning.ipynb)  
